## installation

In [1]:
# !pip install ultralytics

## imports

In [ ]:
import os
import requests
from io import BytesIO
from PIL import Image
import zipfile
import json
import tarfile
import numpy as np
from dataclasses import dataclass, field
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import glob
import cv2
import random
import time
from shutil import copyfile

## Data Transformation

In [ ]:
DATA_DIR = "/content/drive/MyDrive/horse10"

TRAIN_DIR         = f"train"
TRAIN_FOLDER_IMG    = f"images"
TRAIN_FOLDER_LABELS = f"labels"

TRAIN_IMG_PATH   = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_IMG)
TRAIN_LABEL_PATH = os.path.join(DATA_DIR, TRAIN_DIR, TRAIN_FOLDER_LABELS)

VALID_DIR           = f"valid"
VALID_FOLDER_IMG    = f"images"
VALID_FOLDER_LABELS = f"labels"

VALID_IMG_PATH   = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_IMG)
VALID_LABEL_PATH = os.path.join(DATA_DIR, VALID_DIR, VALID_FOLDER_LABELS)

os.makedirs(TRAIN_IMG_PATH, exist_ok=True)
os.makedirs(TRAIN_LABEL_PATH, exist_ok=True)
os.makedirs(VALID_IMG_PATH, exist_ok=True)
os.makedirs(VALID_LABEL_PATH, exist_ok=True)

In [ ]:
import os
import json
from shutil import copyfile
def copy_images_from_json(input_file, output_path):
    with open(input_file, 'r') as file:
        json_data = json.load(file)
    for data in json_data['images']:
        img_file = data["file_name"]
        filename = str(data['id']) + '.png'
        copyfile(os.path.join('/content/drive/MyDrive/horse10/', img_file),
                 os.path.join(output_path, filename))

In [ ]:
# input_file_path = '/content/drive/MyDrive/horse10/annotations/horse10-train-split3.json'
# output_directory_path = '/content/drive/MyDrive/horse10/train/images'
# copy_images_from_json(input_file_path, output_directory_path)

In [ ]:
# input_file_path = '/content/drive/MyDrive/horse10/annotations/horse10-test-split3.json'
# output_directory_path = '/content/drive/MyDrive/horse10/valid/images'
# copy_images_from_json(input_file_path, output_directory_path)

In [ ]:
# with open(input_file_path, 'r') as file:
#         json_data = json.load(file)

In [ ]:
for img_data,annotation_data in  zip(json_data['images'],json_data['annotations']):
  print(img_data.keys())
  print(annotation_data['keypoints'])
  break

dict_keys(['id', 'file_name', 'height', 'width'])
[3.0, 87.3, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
CLASS_ID = 0
def create_yolo_boxes_kpts(img_size, boxes, lm_kpts):

    IMG_W, IMG_H = img_size
    # Modify kpts with visibilities as 1s to 2s.


    # Normalizing factor for bboxes and kpts.
    res_box_array = np.array([IMG_W, IMG_H, IMG_W, IMG_H])
    res_lm_array = np.array([IMG_W, IMG_H])

    # Normalize landmarks in the range [0,1].
    norm_kps_per_img = lm_kpts.copy()
    l=len(norm_kps_per_img)
    i=1
    while i<=l:
      norm_kps_per_img[i-1]=norm_kps_per_img[i-1]/IMG_W
      norm_kps_per_img[i]=norm_kps_per_img[i]/IMG_H
      i=i+3

    # Normalize bboxes in the range [0,1].
    norm_bbox_per_img = boxes / res_box_array

    # Create bboxes coordinates to YOLO.
    # x_c, y_c = x_min + bbox_w/2. , y_min + bbox_h/2.
    yolo_boxes = norm_bbox_per_img.copy()
    yolo_boxes[:2] = norm_bbox_per_img[:2] + norm_bbox_per_img[2:]/2.

    return yolo_boxes, norm_kps_per_img

In [ ]:
def create_yolo_txt_files(json_file, LABEL_PATH):
    with open(json_file, 'r') as file:
        json_data = json.load(file)

    for img_data,annotation_data in  zip(json_data['images'],json_data['annotations']):




        IMAGE_ID = img_data["id"]

        IMG_WIDTH, IMG_HEIGHT = img_data["width"], img_data["height"]

        landmark_kpts  = np.nan_to_num(np.array(annotation_data["keypoints"], dtype=np.float32))
        landmarks_bboxes = np.array(annotation_data["bbox"], dtype=np.float32)
        if len(landmark_kpts)!=66:
          print("false")
        bboxes_yolo, kpts_yolo = create_yolo_boxes_kpts(
                                            (IMG_WIDTH, IMG_HEIGHT),
                                            landmarks_bboxes,
                                            landmark_kpts)

        TXT_FILE = str(IMAGE_ID)+".txt"

        with open(os.path.join(LABEL_PATH, TXT_FILE), "w") as f:

            x_c_norm, y_c_norm, box_width_norm, box_height_norm = round(bboxes_yolo[0],5),\
                                                                  round(bboxes_yolo[1],5),\
                                                                  round(bboxes_yolo[2],5),\
                                                                  round(bboxes_yolo[3],5),\

            kps_flattend = [round(ele,5) for ele in kpts_yolo.flatten().tolist()]
            line = f"{CLASS_ID} {x_c_norm} {y_c_norm} {box_width_norm} {box_height_norm} "
            line+= " ".join(map(str, kps_flattend))
            f.write(line)

In [ ]:
# input_file_path = '/content/drive/MyDrive/horse10/annotations/horse10-train-split3.json'
# create_yolo_txt_files(input_file_path, TRAIN_LABEL_PATH)

In [ ]:
# input_file_path = '/content/drive/MyDrive/horse10/annotations/horse10-test-split3.json'
# create_yolo_txt_files(input_file_path, VALID_LABEL_PATH)

## Model

In [ ]:
@dataclass(frozen=True)
class TrainingConfig:
    DATASET_YAML:   str = "horse-keypoints.yaml"
    MODEL:          str = "yolov8s-pose.pt"
    EPOCHS:         int = 50
    KPT_SHAPE:    tuple = (22,3)
    PROJECT:        str = "Horse_Keypoints"
    NAME:           str = f"{MODEL.split('.')[0]}_{EPOCHS}_epochs"
    CLASSES_DICT:  dict = field(default_factory = lambda:{0 : "horse"})

In [ ]:
@dataclass(frozen=True)
class DatasetConfig:
    BATCH_SIZE:    int   = 32
    CLOSE_MOSAIC:  int   = 10
    MOSAIC:        float = 0.4
    FLIP_LR:       float = 0.0

In [ ]:
train_config = TrainingConfig()
data_config = DatasetConfig()

In [ ]:
current_dir = os.getcwd()

data_dict = dict(
                path      = DATA_DIR,
                train     = os.path.join(TRAIN_DIR, TRAIN_FOLDER_IMG),
                val       = os.path.join(VALID_DIR, VALID_FOLDER_IMG),
                names     = train_config.CLASSES_DICT,
                kpt_shape = list(train_config.KPT_SHAPE),
               )

with open(train_config.DATASET_YAML, "w") as config_file:
    yaml.dump(data_dict, config_file)

In [ ]:
pose_model = model = YOLO(train_config.MODEL)

pose_model.train(data    = train_config.DATASET_YAML,
            epochs       = train_config.EPOCHS,
            batch        = data_config.BATCH_SIZE,
            project      = train_config.PROJECT,
            name         = train_config.NAME,
            close_mosaic = data_config.CLOSE_MOSAIC,
            mosaic       = data_config.MOSAIC,
            fliplr       = data_config.FLIP_LR
           )

100%|██████████| 22.4M/22.4M [00:00<00:00, 205MB/s]


Ultralytics YOLOv8.2.10 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=pose, mode=train, model=yolov8s-pose.pt, data=horse-keypoints.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=Horse_Keypoints, name=yolov8s-pose_50_epochs, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

100%|██████████| 755k/755k [00:00<00:00, 25.6MB/s]


Overriding model.yaml kpt_shape=[17, 3] with kpt_shape=[22, 3]

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1 

100%|██████████| 6.23M/6.23M [00:00<00:00, 114MB/s]


...

      50/50      8.21G     0.2735      0.625     0.1582     0.2027     0.8364         10        640: 100%|██████████| 44/44 [00:32<00:00,  1.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 106/106 [01:22<00:00,  1.28it/s]


                   all       6728       4046      0.588      0.888      0.581      0.531      0.582      0.866       0.56      0.455

50 epochs completed in 1.762 hours.
Optimizer stripped from Horse_Keypoints/yolov8s-pose_50_epochs/weights/last.pt, 23.8MB
Optimizer stripped from Horse_Keypoints/yolov8s-pose_50_epochs/weights/best.pt, 23.8MB

Validating Horse_Keypoints/yolov8s-pose_50_epochs/weights/best.pt...
Ultralytics YOLOv8.2.10 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-pose summary (fused): 187 layers, 11789469 parameters, 0 gradients, 30.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 106/106 [01:27<00:00,  1.21it/s]


                   all       6728       4046      0.588      0.891      0.628      0.573      0.578      0.868      0.606      0.452
Speed: 0.1ms preprocess, 2.8ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to Horse_Keypoints/yolov8s-pose_50_epochs


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b4415116590>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    

In [ ]:
ckpt_path  = os.path.join(train_config.PROJECT, train_config.NAME, "weights", "best.pt")
model_pose = YOLO(ckpt_path)

metrics = model_pose.val()

Ultralytics YOLOv8.2.10 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-pose summary (fused): 187 layers, 11789469 parameters, 0 gradients, 30.9 GFLOPs


val: Scanning /content/drive/.shortcut-targets-by-id/1d9fVKFxu7VuCoVFZafnwSBkWzrkn2_ap/horse10/valid/labels... 4046 images, 2682 backgrounds, 0 corrupt: 100%|██████████| 6728/6728 [29:54<00:00,  3.75it/s]


val: New cache created: /content/drive/.shortcut-targets-by-id/1d9fVKFxu7VuCoVFZafnwSBkWzrkn2_ap/horse10/valid/labels.cache


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 421/421 [01:31<00:00,  4.59it/s]


                   all       6728       4046      0.588      0.891      0.627      0.573      0.578      0.867      0.605      0.451
Speed: 0.2ms preprocess, 5.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/pose/val


## Inference

In [ ]:
colors = [(255, 0, 0),
          (0, 255, 0),
          (0, 0, 255),
          (255, 255, 0),
          (255, 0, 255),
          (0, 255, 255),
          (128, 0, 0),
          (0, 128, 0),
          (0, 0, 128),
          (128, 128, 0),
          (128, 0, 128),
          (0, 128, 128),
          (255, 128, 0),
          (255, 0, 128),
          (128, 255, 0),
          (0, 255, 128),
          (128, 0, 255),
          (0, 128, 255),
          (255, 128, 128),
          (128, 255, 128),
          (128, 128, 255),
          (255, 255, 255)]

In [ ]:
def draw_landmarks(image, landmarks):
    radius = 5
    if image.shape[1] > 1000:
        radius = 2

    color_index = 0
    for kpt_data in landmarks:
        # Assuming kpt_data directly contains the keypoints as [x, y, score,...]
        for j in range(0, len(kpt_data), 3):
            loc_x, loc_y = int(kpt_data[j]), int(kpt_data[j+1])
            color_id = colors[color_index % len(colors)]  # Cycle through colors safely

            # Only draw if the location values are not zero
            if loc_x and loc_y:
                cv2.circle(image, (loc_x, loc_y), radius, color=color_id, thickness=-1, lineType=cv2.LINE_AA)

            color_index += 1

    return image

In [ ]:
def draw_boxes_landmarks(image, detections, color=(0,255,0)):

    font_size = 0.25 + 0.07 * min(image.shape[:2]) / 100
    font_size = max(font_size, 0.5)
    font_size = min(font_size, 0.8)
    text_offset = 3

    thickness = 2
    if (image.shape[1] > 1000):
        thickness = 10

    xmin, ymin, w, h = np.array(detections).astype("int").tolist()
    xmax= xmin+w
    ymax=ymin+h

    image=draw_landmarks(image, first_image_annotations)

    cv2.rectangle(image,
                  (xmin, ymin),
                  (xmax, ymax),
                  color=(0,255,0),
                  thickness=thickness,
                  lineType=cv2.LINE_AA)


    return image